In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [ ]:
tokens = tokenizer.encode('I hated this, absolutely the worst', return_tensors='pt')

In [ ]:
tokens

tensor([[  101,   151, 39487, 10163, 10372,   117, 35925, 10563, 10103, 43060,
           102]])

In [ ]:
result = model(tokens)

In [ ]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.8750,  1.7880, -0.8356, -3.0027, -2.0727]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
result.logits

tensor([[ 4.8750,  1.7880, -0.8356, -3.0027, -2.0727]],
       grad_fn=<AddmmBackward0>)

In [ ]:
torch.argmax(result.logits)

tensor(0)

In [ ]:
int(torch.argmax(result.logits))+1

1

In [ ]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

api_key = 'Your API KEY'
youtube = build('youtube', 'v3', developerKey=api_key)

def get_video_comments(video_id):
    comments = []
    nextPageToken = None
    while True:
        try:
            response = youtube.commentThreads().list(
                part='snippet',
                videoId=video_id,
                maxResults=100,
                pageToken=nextPageToken
            ).execute()

            for item in response['items']:
                comment = item['snippet']['topLevelComment']['snippet']['textOriginal']
                comments.append(comment)

            nextPageToken = response.get('nextPageToken')

            if not nextPageToken:
                break
        except HttpError as e:
            print(f'An HTTP error {e.resp.status} occurred:\n{e.content}')
            break

    return comments

video_id = 'Video ID'

all_comments = get_video_comments(video_id)

print(all_comments[0])

A big reason as to why I bought the s24 as opposed to the other versions is size.


In [ ]:
df = pd.DataFrame(np.array(all_comments), columns=['comments'])

In [ ]:
df['comments'].iloc[0]

'A big reason as to why I bought the s24 as opposed to the other versions is size.'

In [ ]:
def sentiment_score(comments):
  tokens = tokenizer.encode(comments, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df['comments'].iloc[0])

4

In [ ]:
df['sentiment'] = df['comments'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
df

,comments,sentiment
0,A big reason as to why I bought the s24 as opp...,4
1,Any phone bigger than 6 inches won't fit in my...,1
2,If apple made a new small iPhone it would be t...,3
3,My wife is holding onto her iPhone 12 mini for...,5
4,I want that mini coffin.,5
...,...,...
8300,Cool,5
8301,5th,5
8302,3rd,5
8303,Woah I caught it fast,2
